# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries

import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk
from sklearn.pipeline import Pipeline

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to /Users/Tiger/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Tiger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Tiger/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)
categories = df.columns[4:]

X = df[['message']].values[:, 0]
y = df[categories].values

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    #Case Normalization & remove punctuation 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) 
    
    #tokenization methods 
    
    words = word_tokenize(text)
    
    
    
    words = [w for w in words if w not in stopwords.words("english")]

    
    return words  

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25, random_state=35)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def Classification_report(actual, prediction,columns):
    for i in range(0, len(columns)):
        print(columns[i])
        print(" F1_score: {:.4f} % Accuracy: {:.4f}  % Precision: {:.4f} % Recall: {:.4f} ".format(
            f1_score(actual[:, i], prediction[:, i], average='weighted'),
            accuracy_score(actual[:, i], prediction[:, i]),
            precision_score(actual[:, i], prediction[:, i], average='weighted'),
            recall_score(actual[:, i], prediction[:, i], average='weighted')
        ))
        


In [11]:
prediction= pipeline.predict(X_train)

In [12]:
Classification_report(y_train, prediction,categories)

related
 F1_score: 0.9885 % Accuracy: 0.9886  % Precision: 0.9886 % Recall: 0.9886 
request
 F1_score: 0.9858 % Accuracy: 0.9861  % Precision: 0.9862 % Recall: 0.9861 
offer
 F1_score: 0.9981 % Accuracy: 0.9983  % Precision: 0.9983 % Recall: 0.9983 
aid_related
 F1_score: 0.9853 % Accuracy: 0.9853  % Precision: 0.9855 % Recall: 0.9853 
medical_help
 F1_score: 0.9880 % Accuracy: 0.9884  % Precision: 0.9885 % Recall: 0.9884 
medical_products
 F1_score: 0.9916 % Accuracy: 0.9920  % Precision: 0.9920 % Recall: 0.9920 
search_and_rescue
 F1_score: 0.9933 % Accuracy: 0.9937  % Precision: 0.9937 % Recall: 0.9937 
security
 F1_score: 0.9942 % Accuracy: 0.9947  % Precision: 0.9947 % Recall: 0.9947 
military
 F1_score: 0.9957 % Accuracy: 0.9958  % Precision: 0.9958 % Recall: 0.9958 
child_alone
 F1_score: 1.0000 % Accuracy: 1.0000  % Precision: 1.0000 % Recall: 1.0000 
water
 F1_score: 0.9950 % Accuracy: 0.9951  % Precision: 0.9951 % Recall: 0.9951 
food
 F1_score: 0.9951 % Accuracy: 0.9951  % P

In [13]:
y_prediction= pipeline.predict(X_test)

In [14]:
Classification_report(y_test, y_prediction,categories)

related
 F1_score: 0.7860 % Accuracy: 0.7923  % Precision: 0.7858 % Recall: 0.7923 
request
 F1_score: 0.8801 % Accuracy: 0.8923  % Precision: 0.8859 % Recall: 0.8923 
offer
 F1_score: 0.9941 % Accuracy: 0.9960  % Precision: 0.9921 % Recall: 0.9960 
aid_related
 F1_score: 0.7494 % Accuracy: 0.7551  % Precision: 0.7554 % Recall: 0.7551 
medical_help
 F1_score: 0.8937 % Accuracy: 0.9211  % Precision: 0.9007 % Recall: 0.9211 
medical_products
 F1_score: 0.9322 % Accuracy: 0.9510  % Precision: 0.9371 % Recall: 0.9510 
search_and_rescue
 F1_score: 0.9668 % Accuracy: 0.9759  % Precision: 0.9703 % Recall: 0.9759 
security
 F1_score: 0.9757 % Accuracy: 0.9837  % Precision: 0.9679 % Recall: 0.9837 
military
 F1_score: 0.9556 % Accuracy: 0.9677  % Precision: 0.9579 % Recall: 0.9677 
child_alone
 F1_score: 1.0000 % Accuracy: 1.0000  % Precision: 1.0000 % Recall: 1.0000 
water
 F1_score: 0.9493 % Accuracy: 0.9570  % Precision: 0.9540 % Recall: 0.9570 
food
 F1_score: 0.9200 % Accuracy: 0.9286  % P

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 F1_score: 0.9167 % Accuracy: 0.9284  % Precision: 0.9192 % Recall: 0.9284 
fire
 F1_score: 0.9865 % Accuracy: 0.9907  % Precision: 0.9878 % Recall: 0.9907 
earthquake
 F1_score: 0.9608 % Accuracy: 0.9625  % Precision: 0.9609 % Recall: 0.9625 
cold
 F1_score: 0.9774 % Accuracy: 0.9835  % Precision: 0.9819 % Recall: 0.9835 
other_weather
 F1_score: 0.9254 % Accuracy: 0.9474  % Precision: 0.9290 % Recall: 0.9474 
direct_report
 F1_score: 0.8302 % Accuracy: 0.8538  % Precision: 0.8453 % Recall: 0.8538 


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:

parameters = {
           'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}


cv = GridSearchCV(estimator=pipeline, param_grid=parameters,  verbose=3)

In [18]:

cv.fit(X_train,y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.23527616722612146, total= 1.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.2210863594751297, total= 1.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.24153188892279523, total= 1.7min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.240311260299054, total= 1.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.23481843149221848, total= 1.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.24671956057369546, total= 1.7min
[CV] clf__estimator__min_samples_split=2, clf__est

[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.23176685993286542, total= 1.4min
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.22856270979554472, total= 1.4min
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.24275251754653646, total= 1.4min
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.24305767470247178, total= 2.0min
[CV] clf__estimator__min_samples_split=5, c

[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5, score=0.24870308208727496, total= 1.7min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5, score=0.2534330180042722, total= 1.7min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.24397314617027768, total= 1.9min
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.23664937442783032, total= 1.9min
[CV] clf__estimator__min_samples_split

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 194.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__min_df': [1, 5], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [10, 25], 'clf__estimator__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
y_prediction = cv.predict(X_train)


In [20]:
Classification_report(y_train, y_prediction,categories)

related
 F1_score: 0.9859 % Accuracy: 0.9860  % Precision: 0.9859 % Recall: 0.9860 
request
 F1_score: 0.9846 % Accuracy: 0.9849  % Precision: 0.9851 % Recall: 0.9849 
offer
 F1_score: 0.9963 % Accuracy: 0.9970  % Precision: 0.9970 % Recall: 0.9970 
aid_related
 F1_score: 0.9893 % Accuracy: 0.9893  % Precision: 0.9893 % Recall: 0.9893 
medical_help
 F1_score: 0.9829 % Accuracy: 0.9838  % Precision: 0.9840 % Recall: 0.9838 
medical_products
 F1_score: 0.9879 % Accuracy: 0.9886  % Precision: 0.9887 % Recall: 0.9886 
search_and_rescue
 F1_score: 0.9866 % Accuracy: 0.9881  % Precision: 0.9883 % Recall: 0.9881 
security
 F1_score: 0.9875 % Accuracy: 0.9895  % Precision: 0.9896 % Recall: 0.9895 
military
 F1_score: 0.9948 % Accuracy: 0.9950  % Precision: 0.9950 % Recall: 0.9950 
child_alone
 F1_score: 1.0000 % Accuracy: 1.0000  % Precision: 1.0000 % Recall: 1.0000 
water
 F1_score: 0.9950 % Accuracy: 0.9951  % Precision: 0.9951 % Recall: 0.9951 
food
 F1_score: 0.9946 % Accuracy: 0.9946  % P

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
 
         
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(
    AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
    ))
])

parameters = {
    'clf__estimator__learning_rate': [1, 3],
    'clf__estimator__n_estimators': [96, 150]
}

cv2 = GridSearchCV(estimator=pipeline2, param_grid=parameters, verbose=3)



In [28]:

cv2.fit(X_train, y_train)

y_prediction = cv2.predict(X_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=96 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=96, score=0.11458651205370766, total= 3.4min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=96 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.3min remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=96, score=0.09902349710100702, total= 2.7min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=96 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  8.0min remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=96, score=0.10329569728410132, total= 2.7min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=150 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=150, score=0.12709795544705524, total=48.0min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=150 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=150, score=0.11306072627403113, total=46.1min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=150 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=150, score=0.11428135489777236, total=114.7min
[CV] clf__estimator__learning_rate=3, clf__estimator__n_estimators=96 
[CV]  clf__estimator__learning_rate=3, clf__estimator__n_estimators=96, score=0.0, total= 2.0min
[CV] clf__estimator__learning_rate=3, clf__estimator__n_estimators=96 
[CV]  clf__estimator__learning_rate=3, clf__estimator__n_estimators=96, score=0.0, total=

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 248.8min finished


In [29]:
Classification_report(y_train, y_prediction,categories)

related
 F1_score: 0.7016 % Accuracy: 0.6778  % Precision: 0.7898 % Recall: 0.6778 
request
 F1_score: 0.8714 % Accuracy: 0.8630  % Precision: 0.8890 % Recall: 0.8630 
offer
 F1_score: 0.9920 % Accuracy: 0.9892  % Precision: 0.9967 % Recall: 0.9892 
aid_related
 F1_score: 0.7877 % Accuracy: 0.7882  % Precision: 0.7875 % Recall: 0.7882 
medical_help
 F1_score: 0.8962 % Accuracy: 0.8774  % Precision: 0.9305 % Recall: 0.8774 
medical_products
 F1_score: 0.9209 % Accuracy: 0.9007  % Precision: 0.9566 % Recall: 0.9007 
search_and_rescue
 F1_score: 0.9275 % Accuracy: 0.8984  % Precision: 0.9727 % Recall: 0.8984 
security
 F1_score: 0.9299 % Accuracy: 0.8940  % Precision: 0.9812 % Recall: 0.8940 
military
 F1_score: 0.9627 % Accuracy: 0.9543  % Precision: 0.9788 % Recall: 0.9543 
child_alone
 F1_score: 1.0000 % Accuracy: 1.0000  % Precision: 1.0000 % Recall: 1.0000 
water
 F1_score: 0.9587 % Accuracy: 0.9539  % Precision: 0.9695 % Recall: 0.9539 
food
 F1_score: 0.9494 % Accuracy: 0.9464  % P

### 9. Export your model as a pickle file

In [30]:
with open('mlmodel.pkl', 'wb') as file:
    pickle.dump(cv2, file)

In [31]:
with open('mlmodel_1.pkl', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.